In [ ]:
from secretnote.compat.secretflow.device.driver import SFConfigSimulationFullyManaged

secretflow_config = SFConfigSimulationFullyManaged(
    parties=["alice", "bob", "carol", "dave"]
)


In [ ]:
import secretflow

secretflow.shutdown()
secretflow.init(**secretflow_config.dict())


In [ ]:
from secretnote.compat.spu import (
    SPUConfig,
    SPUClusterDef,
    SPUNode,
    SPUProtocolKind,
    SPUFieldType,
    SPURuntimeConfig,
)

spu_config = SPUConfig(
    cluster_def=SPUClusterDef(
        nodes=[
            SPUNode(party="alice", address="localhost:32767"),
            SPUNode(party="bob", address="localhost:32768"),
            SPUNode(party="carol", address="localhost:32769"),
        ],
        runtime_config=SPURuntimeConfig(
            protocol=SPUProtocolKind.ABY3,
            field=SPUFieldType.FM128,
        ),
    ),
)


In [ ]:
alice = secretflow.PYU("alice")
carol = secretflow.PYU("carol")
dave = secretflow.PYU("dave")


In [ ]:
spu = secretflow.SPU(**spu_config.dict())


In [ ]:
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import SERVICE_NAME, Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

from secretnote.instrumentation import ProfilingInstrumentor, MermaidExporter

mermaid = MermaidExporter()

resource = Resource(attributes={SERVICE_NAME: "simulation"})
provider = TracerProvider(resource=resource)
provider.add_span_processor(
    SimpleSpanProcessor(OTLPSpanExporter(endpoint="localhost:4317", insecure=True)),
)
provider.add_span_processor(
    SimpleSpanProcessor(mermaid),
)
trace.set_tracer_provider(provider)


In [ ]:
import secretnote.functional as sfx


def compare(x: float, y: float):
    return x > y


In [ ]:
with ProfilingInstrumentor():
    carol_assets = sfx.use_cleartext(carol)(1000000.0)
    carol_assets = sfx.use_relocation(spu, carol_assets)()
    dave_assets = sfx.use_cleartext(dave)(1000002.0)
    dave_assets = sfx.use_relocation(spu, dave_assets)()
    carol_is_richer = sfx.use_function(spu, carol_assets, dave_assets)(compare)
    carol_is_richer = sfx.use_relocation(alice, carol_is_richer)()
    secretflow.reveal(carol_is_richer)


In [ ]:
print(mermaid.graph())
